## Gestión inicial de vectores en Pinecone

### Inicialización de variables

In [1]:
# Carga de variables de entorno
import os
from dotenv import load_dotenv
load_dotenv()

True

### Carga de documentos que estarán disponibles para el chatbot en la base de datos vectorial Pinecone

In [2]:
# Cargar documentos PDF
from langchain_community.document_loaders import PyPDFDirectoryLoader
import os

def read_docs_from_subdirs(base_directory):
    doc_dict = {}
    # Iterate over items in the base directory
    if os.path.exists(base_directory):
        for name in os.listdir(base_directory):
            # Cada subdirectorio es una persona
            item_path = os.path.join(base_directory, name)
            # Check if it's a directory
            if os.path.isdir(item_path):
                file_loader = PyPDFDirectoryLoader(item_path)
                documents = file_loader.load()
                # Only add if there are documents
                if documents:
                    doc_dict[name] = documents
                    print(f"Name '{name}': {len(documents)} pages loaded")
                else:
                    print(f"No documents found for {name}")
    else:
        print(f"Directory {base_directory} does not exist.")
    return doc_dict

docs_dir = os.getenv("DOCS_DIR") or "./docs"
name_docs_dict = read_docs_from_subdirs(docs_dir)


Name 'Gustavo Vinas': 1 pages loaded
Name 'Rob Otto': 3 pages loaded


### Inicialización de embeddings

In [3]:
### EMBEDDINGS
from langchain_huggingface import HuggingFaceEmbeddings
embedding_model = HuggingFaceEmbeddings(model_name="all-mpnet-base-v2")
embedding_size = embedding_model._client.get_sentence_embedding_dimension()
print(f"Tamaño teórico del embedding: {embedding_size}. Tamaño real del embedding: {len(embedding_model.embed_query('hola'))}.")

Tamaño teórico del embedding: 768. Tamaño real del embedding: 768.


### Fragmentación de documentos con SemanticChunker

In [4]:
from langchain_experimental.text_splitter import SemanticChunker

def chunk_data(docs, embedding_model):
    semantic_chunker = SemanticChunker(embedding_model, breakpoint_threshold_type="percentile")
    semantic_chunks = semantic_chunker.create_documents([d.page_content for d in docs])
    return semantic_chunks

name_chunks = {}
for name, docs in name_docs_dict.items():
    chunks=chunk_data(docs=docs,embedding_model=embedding_model)
    name_chunks[name]=chunks
    print(f"Cantidad de fragmentos para {name}: {len(chunks)}")

Cantidad de fragmentos para Gustavo Vinas: 2
Cantidad de fragmentos para Rob Otto: 6


### Carga inicial de documentos a Pinecone

In [5]:
## CONNECT WITH PINECONE DATABASE
from pinecone import Pinecone, ServerlessSpec
PINECONE_API_KEY=os.getenv("PINECONE_API_KEY")
if not PINECONE_API_KEY:
    raise ValueError("PINECONE_API_KEY not found in environment variables") 

#Connect to Pinecone DB
pc=Pinecone(api_key=PINECONE_API_KEY)
cloud = os.environ.get('PINECONE_CLOUD') or 'aws'
region = os.environ.get('PINECONE_REGION') or 'us-east-1'
spec = ServerlessSpec(cloud=cloud, region=region)
index_name = os.environ.get('PINECONE_INDEX_NAME') or 'ceia-2025-b5-pnl2-tp3'
recreate_index = os.environ.get('PINECONE_RECREATE_INDEX') or False

if recreate_index and index_name in pc.list_indexes().names():
  pc.delete_index(index_name)
  print(f"Index {index_name} borrado.")

# check if index already exists
if index_name not in pc.list_indexes().names():
    # if does not exist, create index
    pc.create_index(
        index_name,
        dimension=embedding_size, # dimensionality of embeddings
        metric='cosine',
        spec=spec
        )
    print(f"Index creado con el nombre: {index_name}.")
else:
    print(f"El index con el nombre {index_name} ya estaba creado.")

Index ceia-2025-b5-pnl2-tp3 borrado.
Index creado con el nombre: ceia-2025-b5-pnl2-tp3.


In [6]:
## UPSERT THE VECTORS IN TO THE PINECONE DATABASE

from langchain_pinecone import PineconeVectorStore
namespace = "documentos"

for name, chunk in name_chunks.items():
    docsearch = PineconeVectorStore.from_documents(
        documents=chunk,
        index_name=index_name,
        embedding=embedding_model, 
        namespace=name
    )
print(f"Upserted values to {index_name} index")

Upserted values to ceia-2025-b5-pnl2-tp3 index


In [7]:
index = pc.Index(index_name)
index_stats = index.describe_index_stats()
namespaces = index_stats.namespaces
for namespace_name in namespaces:
    print(f"Namespace: {namespace_name}")

Namespace: Gustavo Vinas
Namespace: Rob Otto


### Prueba de carga inicial con consultas a la base de datos de Pinecone

In [8]:
vectorstore = PineconeVectorStore(
    index_name=index_name,
    embedding=embedding_model,
    namespace="Gustavo Vinas",
)
retriever=vectorstore.as_retriever()


In [9]:
query = "in which companies did Gustavo Vinas used to work"
vectorstore.similarity_search(query, k=2)

[Document(id='88e898af-0067-43d0-b6dd-067c9132f2af', metadata={}, page_content='INFORMACIÓN PERSONAL \n\ue13d  \nCiudad Autónoma de \nBuenos Aires, C1111ABC \n🏠 \n11-1234-5678 \n\ue13d  \n\ue13d  mimail@micorreo.com \nGustavo Viñas \nIngeniero en Informática \nCuento con más de 20 años de experiencia desarrollando sistemas en diversos proyectos, siendo \nparte de todas las etapas  involucradas, desde análisis  y diseño  hasta puesta en producción  y \nliderazgo. Los desafíos presentados fueron llevados a cabo con profesionalismo, siempre con el \nobjetivo de satisfacer y superar las expectativas del usuario. En los últimos años he participado también en el área de DevOps transformando y automatizando \nel proceso de desarrollo con el objetivo de mejorar la calidad y efectividad de entrega de los \nproductos. Así mismo continúo involucrado en proyectos en el área de sistemas, manteniéndome actualizado \nsobre metodologías y herramientas que forman parte del ciclo de vida del desarrollo 

### Prueba básica de chatbot

In [10]:
from langchain_ollama import ChatOllama

chat = ChatOllama(model="llama3:8b")

In [11]:
from langchain_classic.chains import create_retrieval_chain
from langchain_classic.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

# 1. Crear la cadena que procesa los documentos ("stuff")
prompt = ChatPromptTemplate.from_template("""Answer based in context:\n\n{context}\n\Question: {input}""")
combine_docs_chain = create_stuff_documents_chain(chat, prompt)

# 2. Crear la cadena de recuperación completa
qa_chain = create_retrieval_chain(retriever, combine_docs_chain)

# 3. Ejecutar
result = qa_chain.invoke({"input": query})
print(result["answer"])

According to the provided information, Gustavo Viñas has worked for two companies:

1. Mi Empresa (Mayo 2004 – enero 2020): As an Analista/Desarrollador Senior and Project Manager.
2. Operations Company (Diciembre 2020 – junio 2025): As a Devops professional, responsible for defining and administering pipelines of integration and deployment, among other tasks.

Note that he is currently working as a Socio at Mi Empresa (Enero 2020 – Presente).


In [12]:
query = "does Rob Otto have any hands-on experience."

result = qa_chain.invoke({"input": query})
print(result["answer"])


Based on the context, Gustavo Viñas has extensive hands-on experience in software development and DevOps. He mentions that he has:

* Over 20 years of experience developing systems for various projects, from analysis and design to production and leadership.
* Participated in DevOps transformations and automation of the development process to improve quality and effectiveness of product delivery.
* Experience with various programming languages, frameworks, tools, and technologies, including:
	+ Programming languages: C#, C++, Python, Php, Java
	+ Web development: Js, Html, Bootstrap, Css/sass
	+ .Net (Core, Standard, Framework), EF, Asp.net MVC, API Rest
	+ Databases: SQL, Oracle, MySql, Redis, Mongo
	+ Version control systems: Git
	+ Agile methodologies
	+ Containerization and orchestration: Docker, Kubernetes, Openshift, Helm

He also mentions that he has experience with various projects, including:

* Developing applications for communication between systems using web services (restf

In [13]:
print(result)

{'input': 'does Rob Otto have any hands-on experience.', 'context': [Document(id='88e898af-0067-43d0-b6dd-067c9132f2af', metadata={}, page_content='INFORMACIÓN PERSONAL \n\ue13d  \nCiudad Autónoma de \nBuenos Aires, C1111ABC \n🏠 \n11-1234-5678 \n\ue13d  \n\ue13d  mimail@micorreo.com \nGustavo Viñas \nIngeniero en Informática \nCuento con más de 20 años de experiencia desarrollando sistemas en diversos proyectos, siendo \nparte de todas las etapas  involucradas, desde análisis  y diseño  hasta puesta en producción  y \nliderazgo. Los desafíos presentados fueron llevados a cabo con profesionalismo, siempre con el \nobjetivo de satisfacer y superar las expectativas del usuario. En los últimos años he participado también en el área de DevOps transformando y automatizando \nel proceso de desarrollo con el objetivo de mejorar la calidad y efectividad de entrega de los \nproductos. Así mismo continúo involucrado en proyectos en el área de sistemas, manteniéndome actualizado \nsobre metodologí